Baseball Prediction: 1 - Data Wrangling

In this notebook, we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building. Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file. This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html Towards the bottom of the page there are links for ZIP files containing multiple seasons. Download the zip files containing the 'gamelog' data, decompress it, and then move all of the single season files to a single directory. You will then need the path to that directory for the variable fname below.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
fname = '/Users/antiprotons/Downloads/alldata/gamelogs/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [3]:
df.head(25)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [4]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

df.columns = colnames

df.sample(25)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
1612,20220806,1,Sat,ANA,AL,107,SEA,AL,108,1,2,51,D,NaN,NaN,NaN,SEA03,41507.0,168,010000000,00200000x,34,7,2,0,0,1,0,0,1,0,0,13,0,1,0,0,7,2,2,2,2,0,24,6,0,0,0,0,27,4,0,0,1,2,1,0,0,6,0,7,1,0,0,0,8,4,1,1,1,0,27,2,1,0,1,0,blakr901,Ryan Blakney,cejan901,Nestor Ceja,johna901,Adrian Johnson,hudsm901,Marvin Hudson,NaN,(none),NaN,(none),nevip001,Phil Nevin,servs002,Scott Servais,kirbg001,George Kirby,barrj003,Jaime Barria,swane001,Erik Swanson,frant002,Ty France,barrj003,Jaime Barria,kirbg001,George Kirby,wardt002,Taylor Ward,9,ohtas001,Shohei Ohtani,10,rengl001,Luis Rengifo,4,adelj001,Jo Adell,7,walsj001,Jared Walsh,3,suzuk001,Kurt Suzuki,2,monim001,Mickey Moniak,8,rojaj002,Jose Rojas,5,velaa001,Andrew Velazquez,6,fraza001,Adam Frazier,4,frant002,Ty France,3,winkj002,Jesse Winker,7,santc002,Carlos Santana,10,suare001,Eugenio Suarez,5,crawj002,J.P. Crawford,6,lambj001,Jake Lamb,9,ralec001,Cal Raleigh,2,kelej001,Jarred Kelenic,8,NaN,Y
550,20220518,0,Wed,ARI,NL,39,LAN,NL,37,3,5,51,D,NaN,NaN,NaN,LOS03,35643.0,181,011000100,10040000x,35,9,1,0,0,3,0,0,0,3,0,9,0,0,0,0,8,5,5,5,1,0,24,5,0,0,0,0,33,9,3,0,1,5,0,0,0,0,0,6,2,0,0,0,4,5,3,3,0,0,27,9,0,0,1,0,ortir901,Roberto Ortiz,vanol901,Larry Vanover,rackd901,David Rackley,barbs901,Sean Barber,NaN,(none),NaN,(none),lovut001,Tony Lovullo,robed001,Dave Roberts,buehw001,Walker Buehler,daviz001,Zach Davies,hudsd001,Daniel Hudson,turnj001,Justin Turner,daviz001,Zach Davies,buehw001,Walker Buehler,varsd001,Daulton Varsho,2,luplj001,Jordan Luplow,9,smitp002,Pavin Smith,10,walkc002,Christian Walker,3,rojaj001,Josh Rojas,5,perad001,David Peralta,7,perdg001,Gerardo Perdomo,6,thoma004,Alek Thomas,8,hagej001,Jake Hager,4,bettm001,Mookie Betts,9,freef001,Freddie Freeman,3,turnt001,Trea Turner,6,smitw003,Will Smith,2,muncm001,Max Muncy,5,turnj001,Justin Turner,10,bellc002,Cody Bellinger,8,taylc001,Chris Taylor,7,lux-g001,Gavin Lux,4,NaN,Y
791,20220604,0,Sat,BOS,AL,53,OAK,AL,55,8,0,54,D,NaN,NaN,NaN,OAK01,14796.0,187,003100040,000000000,39,14,5,0,0,8,0,0,0,4,0,8,0,0,1,0,8,3,0,0,0,0,27,5,1,0,0,0,30,4,2,0,0,0,0,0,0,2,0,10,1,0,0,0,5,6,8,8,0,0,27,11,1,0,1,0,libkj901,John Libka,reynj901,Jim Reynolds,tosia901,Alex Tosi,ticht901,Todd Tichenor,NaN,(none),NaN,(none),coraa001,Alex Cora,kotsm001,Mark Kotsay,piven001,Nick Pivetta,blacp001,Paul Blackburn,NaN,(none),vazqc001,Christian

CREATE DATA FRAME CONTAINING THE GAME LOG FOR EVERY GAME OF 'THE MODERN ERA' (1901-PRESENT)

In [6]:
df = pd.DataFrame()
for year in range(1901,2023):
    fname = '/Users/antiprotons/Downloads/alldata/gamelogs/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))

In [7]:
df.shape

(205524, 162)

In [8]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205524 entries, 0 to 2429
Data columns (total 162 columns):
 #    Column                Non-Null Count   Dtype  
---   ------                --------------   -----  
 0    date                  205524 non-null  int64  
 1    dblheader_code        205524 non-null  int64  
 2    day_of_week           205524 non-null  object 
 3    team_v                205524 non-null  object 
 4    league_v              205524 non-null  object 
 5    game_no_v             205524 non-null  int64  
 6    team_h                205524 non-null  object 
 7    league_h              205524 non-null  object 
 8    game_no_h             205524 non-null  int64  
 9    runs_v                205524 non-null  int64  
 10   runs_h                205524 non-null  int64  
 11   outs_total            205524 non-null  int64  
 12   day_night             205524 non-null  object 
 13   completion_info       183 non-null     object 
 14   forfeit_info          43 non-null   

In [9]:
## Calculate a few useful columns

df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)

In [10]:
df.home_victory.mean()

0.5387254043323407

Big picture - have outcome, need features

- Would like to start simple, base features on team results over past n games
- Need to do a bit of "data wrangling"
- First, we need to have each team's games in chronological order
- Then can use pandas 'rolling' functionality to aggregate over windows in the past

- Immediate goal, get batting average, on-base percentage, and slugging percentage
- Need "running values" based on the past, not including the current game

In [11]:
# Let's look at a single team's games

df_reds = df.loc[((df.team_v=='CIN') | (df.team_h=='CIN')), :]
df_reds.shape

(19014, 166)

In [12]:
df_reds.sample(50)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
551,19890523,0,Tue,SLN,NL,42,CIN,NL,42,4,6,51,N,NaN,NaN,NaN,CIN08,21872.0,150.0,000003010,14100000x,33.0,7.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,9.0,1.0,1.0,0.0,0.0,6.0,5.0,6.0,6.0,1.0,0.0,24.0,4.0,1.0,0.0,2.0,0.0,31.0,11.0,4.0,0.0,1.0,6.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,0.0,0.0,3.0,3.0,4.0,4.0,0.0,0.0,27.0,13.0,1.0,0.0,1.0,0.0,westj901,Joe West,mcshj901,John McSherry,hirsm901,Mark Hirschbeck,crawj901,Jerry Crawford,NaN,(none),NaN,(none),herzw101,Whitey Herzog,rosep001,Pete Rose,jackd001,Danny Jackson,kinzm001,Matt Kinzer,franj001,John Franco,larkb001,Barry Larkin,kinzm001,Matt Kinzer,jackd001,Danny Jackson,colev001,Vince Coleman,7,smito001,Ozzie Smith,6,guerp001,Pedro Guerrero,3,brunt001,Tom Brunansky,9,mcgew001,Willie McGee,8,pendt001,Terry Pendleton,5,penat001,Tony Pena,2,oquej001,Jose Oquendo,4,kinzm001,Matt Kinzer,1,saboc001,Chris Sabo,5,grifk001,Ken Griffey,7,larkb001,Barry Larkin,6,davie001,Eric Davis,9,oneip001,Paul O'Neill,8,benzt001,Todd Benzinger,3,harrl001,Lenny Harris,4,reedj001,Jeff Reed,2,jackd001,Danny Jackson,1,NaN,Y,1989,2,1,10,198905230
210,19320516,0,Mon,NY1,NL,22,CIN,NL,32,2,6,51,D,NaN,NaN,NaN,CIN07,3100.0,104.0,101000000,00000600x,30.0,3.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,1.0,2.0,6.0,6.0,0.0,0.0,24.0,11.0,0.0,0.0,2.0,0.0,30.0,8.0,2.0,0.0,0.0,6.0,0.0,0.0,1.0,3.0,1.0,3.0,0.0,0.0,2.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,27.0,11.0,1.0,0.0,0.0,0.0,klemb901,Bill Klem,stard901,Dolly Stark,NaN,(none),donnc901,Charles Donnelly,NaN,(none),NaN,(none),mcgrj101,John McGraw,howld101,Dan Howley,lucar102,Red Lucas,fitzf101,Freddie Fitzsimmons,NaN,(none),heatm101,Mickey Heath,fitzf101,Freddie Fitzsimmons,lucar102,Red Lucas,crith101,Hughie Critz,4,koenl101,Len Koenecke,7,lindf101,Freddie Lindstrom,8,terrb101,Bill Terry,3,ott-m101,Mel Ott,9,jackt101,Travis Jackson,6,vergj101,Johnny Vergez,5,hogas101,Shanty Hogan,2,fitzf101,Freddie Fitzsimmons,1,doutt101,Taylor Douthit,8,grang102,George Grantham,4,hermb102,Babe Herman,9,hafec101,Chick Hafey,7,gilbw101,Wally Gilbert,5,heatm101,Mickey Heath,3,manic101,Clyde Manion,2,durol101,Leo Durocher,6,lucar102,Red Lucas,1,NaN,Y,1932,4,1,8,193205160
934,19570818,0,Sun,CIN,NL,116,CHN,NL,116,2,8,51,D,NaN,NaN,NaN,CHI11,27100.0,148.0,002000000,00024101x,34.0,8.0,4.0,0.0,0.0,2.0,0.0,0.0,

In [13]:
# Write a function to create a team-specific data frame, given the team
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)
df_reds = create_team_df('CIN')
df_reds.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1946089,19460727,1,Sat,CIN,NL,89,3,51,D,NaN,NaN,NaN,NYC14,0.0,110.0,2001,34.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,6.0,0.0,0.0,0.0,0.0,7.0,2.0,4.0,4.0,0.0,0.0,24.0,10.0,3.0,0.0,1.0,0.0,rearb901,Beans Reardon,goetl901,Larry Goetz,NaN,(none),jordl901,Lou Jorda,NaN,(none),NaN,(none),mckeb101,Bill McKechnie,schuh103,Hal Schumacher,heuse101,Ed Heusser,thomj102,Junior Thompson,NaN,(none),heuse101,Ed Heusser,clayd101,Dain Clay,8,freyl101,Lonny Frey,9,adamb103,Bobby Adams,4,hattg101,Grady Hatton,5,haasb102,Bert Haas,3,muelr101,Ray Mueller,2,lukoe101,Eddie Lukon,7,corbc101,Claude Corbitt,6,heuse101,Ed Heusser,1,NaN,Y,1946,2,1,8,194607271,0,NY1,5475.0,1324.0,227.0,26.0,77.0,449.0,561.0,74.0,45.0,158.0,0.241826,0.299291,0.334977,0.634268,1007.0,235.0,29.0,4.0,12.0,102.0,92.0,17.0,7.0,29.0,0.233366,0.303877,0.305859,0.609736
1995071,19950715,0,Sat,CIN,NL,71,4,54,N,NaN,NaN,NaN,CHI11,39130.0,173.0,010100101,38.0,12.0,3.0,0.0,1.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,0.0,10.0,2.0,3.0,3.0,0.0,0.0,27.0,9.0,0.0,0.0,2.0,0.0,darlg901,Gary Darling,hirsm901,Mark Hirschbeck,pullf901,Frank Pulli,westj901,Joe West,NaN,(none),NaN,(none),johnd105,Davey Johnson,schop001,Pete Schourek,navaj001,Jaime Navarro,branj001,Jeff Brantley,waltj001,Jerome Walton,schop001,Pete Schourek,howat001,Thomas Howard,8,larkb001,Barry Larkin,6,gantr001,Ron Gant,7,sandr002,Reggie Sanders,9,morrh001,Hal Morris,3,santb001,Benito Santiago,2,boonb002,Bret Boone,4,branj002,Jeff Branson,5,schop001,Pete Schourek,1,NaN,Y,1995,-1,0,7,199507150,0,CHN,5634.0,1579.0,300.0,53.0,185.0,557.0,854.0,208.0,68.0,105.0,0.280263,0.345017,0.450834,0.795851,1014.0,278.0,60.0,4.0,43.0,97.0,166.0,48.0,13.0,20.0,0.274162,0.337534,0.468442,0.805976
1926141,19260911,0,Sat,CIN,NL,141,10,54,D,NaN,NaN,NaN,PIT06,14000.0,108.0,400401001,38.0,13.0,3.0,1.0,0.0,10.0,2.0,0.0,2.0,5.0,0.0,1.0,0.0,0.0,1.0,0.0,10.0,1.0,6.0,6.0,0.0,0.0,27.0,19.0,0.0,0.0,2.0,0.0,riglc901,Cy Rigler,wilsf901,Frank Wilson,NaN,(none),morac102,Charlie Moran,NaN,(none),NaN,(none),hendj101,Jack Hendricks,maysc101,Carl Mays,hillc102,Carmen Hill,NaN,(none),pippw101,Wally Pipp,maysc101,Carl Mays,chric101,Cuckoo Christensen,9,zitzb101,Billy Zitzmann,7,rouse101,Edd Roush,8,pippw101,Wally Pipp,3,dresc101,Chuck Dressen,5,crith101,Hughie Critz,4,hargb101,Bubbles Hargrave,2,fordh101,Hod Ford,6,maysc101,Carl Mays,1,NaN,Y,1926,-4,0,16,192609110,0,PIT,5458.0,1581.0,237.0,127.0,35.0,465.0,773.0,70.0,89.0,198.0,0.289667,0.345433,0.398864,

In [14]:
# Make a dictionary that maps a team name to it's data frame
# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [15]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features
## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%2500==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000
147500
150000
152500
155000
157500
160000
162500
165000
167500
170000
172500
175000
177500
180000
182500
185000
187500
190000
192500
195000
197500
200000
202500
205000


In [16]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v


df.shape

df.sample(25)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
456,19870517,0,Sun,CLE,AL,37,DET,AL,35,4,8,51,D,NaN,NaN,NaN,DET04,20824.0,165.0,010010020,20013020x,33.0,7.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,6.0,0.0,5.0,0.0,0.0,2.0,0.0,8.0,3.0,6.0,6.0,0.0,0.0,24.0,5.0,2.0,0.0,1.0,0.0,28.0,7.0,2.0,0.0,2.0,8.0,0.0,2.0,1.0,6.0,1.0,6.0,2.0,1.0,1.0,1.0,6.0,2.0,2.0,2.0,0.0,0.0,27.0,12.0,2.0,0.0,2.0,0.0,clara901,Al Clark,cousd901,Derryl Cousins,evanj901,Jim Evans,bremn901,Nick Bremigan,NaN,(none),NaN,(none),corrp102,Pat Corrales,andes101,Sparky Anderson,terrw001,Walt Terrell,carls001,Steve Carlton,NaN,(none),hernl001,Larry Herndon,carls001,Steve Carlton,terrw001,Walt Terrell,butlb001,Brett Butler,8,bernt001,Tony Bernazard,4,franj002,Julio Franco,10,cartj001,Joe Carter,9,tablp001,Pat Tabler,3,hallm001,Mel Hall,7,jacob001,Brook Jacoby,5,snydc001,Cory Snyder,6,dempr001,Rick Dempsey,2,whitl001,Lou Whitaker,4,heatm001,Mike Heath,2,gibsk001,Kirk Gibson,10,hernl001,Larry Herndon,7,harpt001,Terry Harper,9,lemoc001,Chet Lemon,8,evand001,Darrell Evans,3,broot001,Tom Brookens,6,coled001,Darnell Coles,5,NaN,Y,1987,4,1,12,198705170,0.263462,0.276784,0.340666,0.330382,0.422299,0.426421,0.762965,0.756803,143.0,145.0,64.0,55.0,107.0,144.0,0.256238,0.228659,0.334192,0.295265,0.396353,0.364837,0.730546,0.660102,14.0,22.0,16.0,11.0,26.0,27.0
754,20170529,0,Mon,BOS,AL,50,CHA,AL,50,4,5,51,D,NaN,NaN,NaN,CHI12,27148.0,196.0,100210000,00300020x,30.0,4.0,2.0,0.0,1.0,4.0,0.0,1.0,1.0,4.0,0.0,10.0,0.0,0.0,0.0,0.0,5.0,4.0,5.0,5.0,0.0,0.0,24.0,9.0,0.0,0.0,1.0,0.0,28.0,5.0,1.0,1.0,1.0,5.0,0.0,0.0,2.0,3.0,0.0,10.0,1.0,0.0,1.0,0.0,4.0,5.0,4.0,4.0,0.0,0.0,27.0,6.0,0.0,0.0,0.0,0.0,sches901,Stu Scheurwater,reynj901,Jim Reynolds,gibsg901,Greg Gibson,barrl901,Lance Barrett,NaN,(none),NaN,(none),farrj001,John Farrell,rentr001,Rich Renteria,minaj001,Juan Minaya,barnm001,Matt Barnes,robed002,David Robertson,cabrm002,Melky Cabrera,pricd001,David Price,holmd002,David Holmberg,bettm001,Mookie Betts,9,pedrd001,Dustin Pedroia,4,bogax001,Xander Bogaerts,6,ramih003,Hanley Ramirez,10,younc004,

In [17]:
df.to_csv('df_bp1.csv', index=False)